# Probability & Statistics for ML — Companion Notebook

Use this notebook alongside `probability_statistics_for_ml.md`. Each section includes runnable code, short exercises, and reference links to official docs.

**Note:** Some packages may be needed: `numpy`, `scipy`, `statsmodels`, `scikit-learn`, `matplotlib`. Install them in your environment as required.


## 1) Probability foundations: LTP & Bayes
- Law of Total Probability and Bayes’ rule appear everywhere in ML—Naive Bayes, filtering, inference.

Refs: SciPy/Stats docs and standard probability texts.


In [ ]:

import numpy as np

rng = np.random.default_rng(0)

# Toy disease screening example for Bayes rule
p_disease = 0.01
sens = 0.95   # P(+ | disease)
spec = 0.98   # P(- | no disease)

N = 1_000_000
d = rng.random(N) < p_disease
test_plus = np.empty(N, dtype=bool)
test_plus[d] = rng.random(d.sum()) < sens
test_plus[~d] = rng.random((~d).sum()) < (1-spec)

# Posterior P(disease | +) via simulation
post_sim = (d & test_plus).sum() / test_plus.sum()

# Analytical via Bayes:
# P(d|+) = P(+|d)P(d) / [P(+|d)P(d) + P(+|~d)P(~d)]
post_bayes = (sens*p_disease) / (sens*p_disease + (1-spec)*(1-p_disease))
post_sim, post_bayes


**Exercise 1 (Bayes):** Change prevalence, sensitivity, and specificity; observe how the posterior \(P(d\mid +)\) reacts.
Add a small function that returns the posterior for arbitrary inputs.


## 2) Core distributions & the law of rare events
**Poisson as a Binomial limit:** With \(n\to\infty, p\to 0, np=\lambda\), Binomial converges to Poisson(\(\lambda\)).
Refs: Poisson distribution background.


In [ ]:

import numpy as np
from math import factorial

def pmf_poisson(lmbda, k):
    return np.exp(-lmbda) * (lmbda**k) / factorial(k)

def pmf_binom_approx(lmbda, k, n=10_000):
    p = lmbda/n
    # Use log-sum trick for stability for large n via logs if desired; here k is small for demo
    from math import comb
    return comb(n, k) * (p**k) * ((1-p)**(n-k))

lam = 3.2
k_vals = np.arange(0, 12)
po = np.array([pmf_poisson(lam, k) for k in k_vals])
bi = np.array([pmf_binom_approx(lam, k, n=50_000) for k in k_vals])
np.column_stack([k_vals, po, bi, np.abs(po-bi)])


**Exercise 2 (memorylessness):** Simulate exponential waiting times and empirically verify \(P(X>s+t\mid X>s)=P(X>t)\).


## 3) Estimation: CIs for means & proportions
### Welch's t-interval for mean difference
Ref: `scipy.stats.ttest_ind` (Welch via `equal_var=False`).

In [ ]:

import numpy as np
from scipy import stats

rng = np.random.default_rng(42)
x = rng.normal(0.0, 1.2, 80)
y = rng.normal(0.4, 1.7, 120)

t_stat, p_val = stats.ttest_ind(x, y, equal_var=False)

nx, ny = len(x), len(y)
vx, vy = x.var(ddof=1), y.var(ddof=1)
se = np.sqrt(vx/nx + vy/ny)
df = (vx/nx + vy/ny)**2 / ((vx**2/((nx**2)*(nx-1))) + (vy**2/((ny**2)*(ny-1))))
delta = stats.t.ppf(0.975, df) * se
ci = (x.mean()-y.mean()-delta, x.mean()-y.mean()+delta)
t_stat, p_val, ci


### Proportion CIs (Wilson, Clopper–Pearson)
Ref: `statsmodels.stats.proportion.proportion_confint` (methods: `'wilson'`, `'beta'`).

In [ ]:

from statsmodels.stats.proportion import proportion_confint

count, n = 37, 200
ci_wilson = proportion_confint(count, n, method="wilson")
ci_exact  = proportion_confint(count, n, method="beta")  # Clopper–Pearson
ci_wilson, ci_exact


**Exercise 3 (coverage):** Monte‑carlo compare Wilson vs. Wald coverage for small \(p\) (e.g., \(p=0.05\), \(n=50\)).


## 4) Resampling: bootstrap & permutation tests
Refs: SciPy `bootstrap` and `permutation_test`.

In [ ]:

from scipy import stats
rng = np.random.default_rng(0)
sample = rng.normal(loc=0.0, scale=1.0, size=500)

# Bootstrap CI for the mean (BCa)
res = stats.bootstrap((sample,), np.mean, n_resamples=10_000, method="BCa")
res.confidence_interval


In [ ]:

# Permutation test on difference in means
rng = np.random.default_rng(1)
a = rng.lognormal(mean=0.0, sigma=1.0, size=60)
b = rng.lognormal(mean=0.2, sigma=1.2, size=80)

def stat_fn(x, y, axis):
    return x.mean(axis=axis) - y.mean(axis=axis)

perm = stats.permutation_test((a, b), stat_fn, n_resamples=100_000,
                              alternative="two-sided", random_state=rng)
perm.pvalue


**Exercise 4 (robust stat):** Redo the permutation test using the median as the statistic.


## 5) A/B design: power & multiple testing
Refs: `statsmodels.stats.power.TTestIndPower` / `NormalIndPower`; `statsmodels.stats.multitest.fdrcorrection`.

In [ ]:

from statsmodels.stats.power import TTestIndPower, NormalIndPower
from statsmodels.stats.multitest import fdrcorrection
import numpy as np

# Power / sample size
t_power = TTestIndPower().power(effect_size=0.3, nobs1=200, alpha=0.05)
n_per_arm = TTestIndPower().solve_power(effect_size=0.3, power=0.8, alpha=0.05)

# FDR across many p-values (Benjamini–Hochberg)
pvals = np.array([0.03, 0.20, 0.002, 0.08, 0.049])
reject, p_adj = fdrcorrection(pvals, alpha=0.05, method="indep")
t_power, n_per_arm, reject, p_adj


**Exercise 5 (proportions):** Convert a desired absolute lift in conversion (p0→p1) into Cohen’s *h* and compute required \(n\) via `NormalIndPower`.


## 6) Classifier metrics: ROC/PR, calibration, Brier
Refs: scikit‑learn `roc_auc_score`, `precision_recall_curve`, `average_precision_score`, `calibration_curve`, `brier_score_loss`.


In [ ]:

import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

X, y = make_classification(n_samples=4000, n_features=20, weights=[0.85,0.15], random_state=0)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

clf = LogisticRegression(max_iter=1000).fit(Xtr, ytr)
p_raw = clf.predict_proba(Xte)[:,1]
auc = roc_auc_score(yte, p_raw)
precision, recall, thr = precision_recall_curve(yte, p_raw)
ap = average_precision_score(yte, p_raw)
brier_raw = brier_score_loss(yte, p_raw)

cal = CalibratedClassifierCV(base_estimator=clf, method='isotonic', cv=3).fit(Xtr, ytr)
p_cal = cal.predict_proba(Xte)[:,1]
brier_cal = brier_score_loss(yte, p_cal)

auc, ap, brier_raw, brier_cal


In [ ]:

# Reliability diagram (plot)
import matplotlib.pyplot as plt

for p, lbl in [(p_raw, "raw"), (p_cal, "isotonic")]:
    prob_true, prob_pred = calibration_curve(yte, p, n_bins=10, strategy="quantile")
    plt.plot(prob_pred, prob_true, marker='o', label=f"{lbl} (Brier={brier_score_loss(yte,p):.3f})")
plt.plot([0,1],[0,1],'--',alpha=.5,label='perfect')
plt.xlabel("Predicted probability"); plt.ylabel("Empirical frequency")
plt.legend(); plt.tight_layout()


**Exercise 6 (imbalance):** Change class weights and observe the effect on PR curve and calibration.


## 7) Monitoring: covariate shift (KS) and PSI
Refs: SciPy `ks_2samp`; industry references for PSI definition and thresholds.


In [ ]:

import numpy as np
from scipy.stats import ks_2samp

def psi(expected, actual, bins=10):
    qs = np.quantile(expected, np.linspace(0,1,bins+1))
    ex, _ = np.histogram(expected, bins=qs)
    ac, _ = np.histogram(actual,   bins=qs)
    ex = np.where(ex==0, 1, ex); ac = np.where(ac==0, 1, ac)
    ex_pct, ac_pct = ex/ex.sum(), ac/ac.sum()
    return np.sum((ac_pct - ex_pct) * np.log(ac_pct / ex_pct))

rng = np.random.default_rng(0)
train = rng.normal(0,1, 4000)
live  = rng.normal(0.2,1.1, 4000)

ks_stat, ks_p = ks_2samp(train, live)
psi_val = psi(train, live)
ks_stat, ks_p, psi_val


**Exercise 7 (alerting):** Choose sensible PSI and KS thresholds for your domain and simulate weekly drift scenarios. Log decisions.


## References
- SciPy bootstrap: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html
- SciPy permutation_test: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.permutation_test.html
- StatsModels proportion_confint: https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html
- StatsModels power: https://www.statsmodels.org/stable/generated/statsmodels.stats.power.TTestIndPower.html
- scikit‑learn calibration_curve: https://scikit-learn.org/stable/modules/generated/sklearn.calibration.calibration_curve.html
- scikit‑learn ROC AUC: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
- scikit‑learn precision_recall_curve: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html
- SciPy ks_2samp: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html
- PSI background (SAS/IBM): https://support.sas.com/resources/papers/proceedings10/288-2010.pdf , https://www.ibm.com/think/topics/model-drift
